In [3]:
from typing import List
import aiohttp
import asyncio
from tortoise import Tortoise, run_async
from models.models import CurrencyApiKey
from data import config

In [25]:
async def set_active_token_currency():
    await Tortoise.init(config.TORTOISE_ORM)


    list_key = await CurrencyApiKey.all().order_by("created_at")
    set_next_key = False
    for key in list_key:
        if key.is_active:
            set_next_key = True
            key.is_active = False
            await key.save()
            continue
        
        if set_next_key is True:
            key.is_active = True
            await key.save()
            break

    if (await CurrencyApiKey.get_or_none(is_active=True)) is None:
        first_key = list_key[0]
        first_key.is_active = True
        await first_key.save()

In [33]:
await set_active_token_currency()

In [37]:
async def get_all_currency(currency: List[str]):
    await Tortoise.init(config.TORTOISE_ORM)
    res = None
    for i in range(10):
        token = await CurrencyApiKey.get(is_active=True)
        url = "https://api.currencyapi.com/v3/latest"
        params = {"apikey": token.key}
        async with aiohttp.ClientSession() as session:
            async with session.get(url=url, params=params) as resp:
                if resp.status == 429:
                    await set_active_token_currency()
                    continue
                elif resp.status == 200:
                    res = await resp.json()
                    break
    if res is None:
        pass
    else:
        return res

In [39]:
res = await get_all_currency(['rub'])
print(res)

{'meta': {'last_updated_at': '2022-07-11T23:59:59Z'}, 'data': {'ADA': {'code': 'ADA', 'value': 0.43677}, 'AED': {'code': 'AED', 'value': 3.67317}, 'AFN': {'code': 'AFN', 'value': 88.15279}, 'ALL': {'code': 'ALL', 'value': 116.08756}, 'AMD': {'code': 'AMD', 'value': 410.97025}, 'ANG': {'code': 'ANG', 'value': 1.80554}, 'AOA': {'code': 'AOA', 'value': 429.66647}, 'ARS': {'code': 'ARS', 'value': 127.35057}, 'AUD': {'code': 'AUD', 'value': 1.48306}, 'AVAX': {'code': 'AVAX', 'value': 17.42552}, 'AWG': {'code': 'AWG', 'value': 1.79755}, 'AZN': {'code': 'AZN', 'value': 1.70005}, 'BAM': {'code': 'BAM', 'value': 1.93745}, 'BBD': {'code': 'BBD', 'value': 2.0228}, 'BDT': {'code': 'BDT', 'value': 93.68463}, 'BGN': {'code': 'BGN', 'value': 1.9482}, 'BHD': {'code': 'BHD', 'value': 0.377}, 'BIF': {'code': 'BIF', 'value': 2062.45208}, 'BMD': {'code': 'BMD', 'value': 1.00001}, 'BNB': {'code': 'BNB', 'value': 228.57819}, 'BND': {'code': 'BND', 'value': 1.40545}, 'BOB': {'code': 'BOB', 'value': 6.89749},

In [3]:
async def create_currency_key(key: str, email: str, password: str):
    await Tortoise.init(config.TORTOISE_ORM)
    await CurrencyApiKey.create(key=key,
                                email=email,
                                password=password)

In [31]:
key = "V3UAxDAeqFqViOKSRViSFfLHb13MorlyEDesVpCZ"
email = "rickrosssss@proton.me"
password = "otefmmn4ik1JNsXR96y8"
await create_currency_key(key=key, email=email, password=password)

In [33]:
x = 1022.

def trim_float(digit: float):
    digit_split = str(digit).split(".")
    if len(digit_split[1]) > 2:
        digit_split[1] = digit_split[1][:len(digit_split[1]) - (len(digit_split[1]) - 2)]

    if digit_split[1] == "0":
        return int(digit_split[0])
    return float(".".join(digit_split))

trim_float(x) 

1022

In [1]:
async def get_currency_ton():
    # ?ids=the-open-network&vs_currencies=usd
    url = "https://api.coingecko.com/api/v3/simple/price"
    params = {'ids': "the-open-network", "vs_currencies": "usd"}
    async with aiohttp.ClientSession() as session:
        async with session.get(url=url, params=params) as resp:
            res = await resp.json()
            return res['the-open-network']['usd']

In [4]:
await get_currency_ton()

0.951918